In [1]:
!pip install azure-storage-blob
!pip install textblob

  Attempting uninstall: typing-extensions
    Found existing installation: typing-extensions 3.10.0.2
    Uninstalling typing-extensions-3.10.0.2:
      Successfully uninstalled typing-extensions-3.10.0.2


In [2]:
!pip install feedparser

  Created wheel for sgmllib3k: filename=sgmllib3k-1.0.0-py3-none-any.whl size=6065 sha256=41e3c4dc195afe22f83fab205b8c5360157abed5d42e4214f51b061ef6c05f62
  Stored in directory: c:\users\rushi\appdata\local\pip\cache\wheels\65\7a\a7\78c287f64e401255dff4c13fdbc672fed5efbfd21c530114e1
Successfully built sgmllib3k


In [3]:
!pip install newspaper3k

  Created wheel for tinysegmenter: filename=tinysegmenter-0.3-py3-none-any.whl size=13552 sha256=726f373ecf980ef883c7f096e21f3ec970254da1a1d475fede3f73a3d7a681cb
  Stored in directory: c:\users\rushi\appdata\local\pip\cache\wheels\94\ad\df\a2a01300cea47d5695f242f7e925a805970106fd9e4b151468
  Created wheel for feedfinder2: filename=feedfinder2-0.0.4-py3-none-any.whl size=3356 sha256=ed7205e0c6611e8da09c8ffd1c2bf762212fa686e575891b9183841bf08016c7
  Stored in directory: c:\users\rushi\appdata\local\pip\cache\wheels\43\4a\c2\61a371b2524ac90805391c660d8dc4505705297f25e2b85a5d
  Created wheel for jieba3k: filename=jieba3k-0.35.1-py3-none-any.whl size=7398405 sha256=d60cde45d2ca958c12e5f677425cf9d9db88ff3bce50eee8a6d251c12c5c8807
  Stored in directory: c:\users\rushi\appdata\local\pip\cache\wheels\c2\22\59\8214a8d6357e9f540ce1f37f9a4362b6156b4ca81b37f1945f
Successfully built tinysegmenter feedfinder2 jieba3k


In [39]:
!python -m textblob.download_corpora

Finished.


[nltk_data] Downloading package brown to
[nltk_data]     C:\Users\rushi\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\brown.zip.
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\rushi\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\rushi\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\rushi\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping taggers\averaged_perceptron_tagger.zip.
[nltk_data] Downloading package conll2000 to
[nltk_data]     C:\Users\rushi\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\conll2000.zip.
[nltk_data] Downloading package movie_reviews to
[nltk_data]     C:\Users\rushi\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\movie_reviews.zip.


In [4]:
!pip install spacy

  Using cached colorama-0.4.6-py2.py3-none-any.whl (25 kB)
  Attempting uninstall: colorama
    Found existing installation: colorama 0.4.4
    Uninstalling colorama-0.4.4:
      Successfully uninstalled colorama-0.4.4


In [11]:
!python -m spacy download en_core_web_lg


[+] Download and installation successful
You can now load the package via spacy.load('en_core_web_lg')


In [13]:
nlp = spacy.load('en_core_web_lg')  # Load large model


In [15]:
import feedparser
from newspaper import Article
from textblob import TextBlob
import pandas as pd
import spacy
from azure.storage.blob import BlobServiceClient, BlobClient, ContainerClient

# Load spaCy language model
nlp = spacy.load('en_core_web_lg')  # Using medium-sized model


In [18]:
!pip install aiohttp


In [35]:
# Function to extract articles from an RSS feed and perform sentiment analysis
def extract_articles_from_rss(feed_url):
    try:
        feed = feedparser.parse(feed_url)
        articles = []
        for entry in feed.entries:
            url = entry.link
            article = Article(url)
            article.download()
            article.parse()

            title = article.title.strip()
            link = url.strip()
            content = article.text.strip()

            blob = TextBlob(content)
            sentiment_score = blob.sentiment.polarity

            # Extract image link (if present)
            image_link = article.top_image if hasattr(article, 'top_image') else None

            # Extract publication year
            publication_year = None
            if 'published_parsed' in entry:
                publication_year = entry.published_parsed.tm_year

             # Perform Keyword Extraction using TextBlob
            keyword_blob = TextBlob(content)
            keywords = keyword_blob.noun_phrases

            # Perform Entity Recognition using spaCy
            doc = nlp(content)
            entities = [(ent.text, ent.label_) for ent in doc.ents]

            articles.append({
                'Title': title,
                'Link': link,
                'Content': content,
                'Sentiment Score': sentiment_score,
                'Image Link': image_link,
                'Publication Year': publication_year,
                'Keywords': keywords,
                'Entities': entities
            })

            # Print the article details
            print(f"Title: {title}")
            print(f"Link: {link}")

        return articles
    except Exception as e:
        print(f"An error occurred while processing {feed_url}: {e}")
        return []

In [40]:
rss_feeds = {
    'https://rss.nytimes.com/services/xml/rss/nyt/Travel.xml': 'Travel',
    'https://rss.nytimes.com/services/xml/rss/nyt/Arts.xml': 'Entertainment',
    'http://rss.cnn.com/rss/cnn_showbiz.rss': 'Entertainment'
}

# Asynchronously fetch and process articles for a single section
all_articles = []
for feed_url, section in rss_feeds.items():
    print(f"Fetching articles from {section} section:")
    articles = extract_articles_from_rss(feed_url)
    for article in articles:
        article['Section'] = section
    all_articles.extend(articles)

Fetching articles from Travel section:


C:\Users\rushi\anaconda3\lib\site-packages\nltk\tag\sequential.py:179: RuntimeWarning: coroutine 'extract_articles_from_rss' was never awaited
  tokens, tags = zip(*sentence)


Title: Dancing Till Dawn on the Albanian Riviera
Link: https://www.nytimes.com/2023/07/31/travel/albania-kala-music-festival.html
Title: A High-Water Year for River Rafting
Link: https://www.nytimes.com/2023/07/28/travel/white-water-rafting-rivers.html
Title: Travel 101: Our Favorite Hacks, Hints and How-Tos
Link: https://www.nytimes.com/explain/2023/07/26/travel/travel-hacks-tips
Title: The Wind, the Water, the Islands: Exploring Stockholm’s Archipelago
Link: https://www.nytimes.com/2023/07/25/travel/stockholm-archipelago.html
Title: In Search of a Crowd-Free Summer Getaway? Try These 6 Hotels.
Link: https://www.nytimes.com/2023/07/24/travel/summer-getaways-hotels.html
Title: Traveling to Europe? Here’s What You Need to Know About the Heat Wave.
Link: https://www.nytimes.com/2023/07/19/travel/heat-europe-vacation-tourism-safety.html
Title: It’s Been a Hellish Summer for the Mediterranean. And It’s Not Over.
Link: https://www.nytimes.com/2023/07/29/world/europe/italy-greece-wildfires-h

In [43]:
# Create a pandas DataFrame from the extracted data
df = pd.DataFrame(all_articles)

def send_to_azure_blob_storage(df, container_name, blob_name, connection_string):
    try:
        blob_service_client = BlobServiceClient.from_connection_string(connection_string)
        container_client = blob_service_client.get_container_client(container_name)

        blob_data = df.to_csv(index=False)
        blob_client = container_client.get_blob_client(blob_name)
        blob_client.upload_blob(blob_data)

        print(f"Uploaded to Azure Blob Storage in container '{container_name}' as '{blob_name}'.")
    except Exception as e:
        print(f"An error occurred during upload to Azure Blob Storage: {e}")

# Replace the following placeholders with your Azure Blob Storage information
container_name = 'assn4'
blob_name = 'newdata.csv'
connection_string = 'DefaultEndpointsProtocol=https;AccountName=assn4strg;AccountKey=KSeVmhAXiNTMvX5ZFuSWRA/B4BW2erjJ+8d59hkMmmH7LwTX6RTWczx3Yf0vaUr9iGuRprgjZx7F+AStvI4h5Q==;EndpointSuffix=core.windows.net'

# Send the DataFrame as CSV to Azure Blob Storage
send_to_azure_blob_storage(df, container_name, blob_name, connection_string)

Uploaded to Azure Blob Storage in container 'assn4' as 'newdata.csv'.
